In [1]:
import json
import pandas as pd

In [2]:
methods = ['gendered', 'efga']
functions = ['rastrigin', 'ackley', 'griewank', 'schwefel']

In [3]:
import os

In [4]:
plot_dir = 'img/Iteration 3/'
log_dir = 'log/'
files = [os.path.join(plot_dir, x) for x in os.listdir(plot_dir)]
extract_ts = lambda x: '.'.join(x.split('_')[-1].split('.')[:-1])
timestamps = list(map(extract_ts, files))

In [5]:
log_files = [os.path.join(log_dir, x) for x in os.listdir(log_dir)]
log_files = list(filter(lambda x: extract_ts(x) in timestamps, log_files))

In [6]:
len(timestamps), len(log_files)

(40, 80)

In [7]:
csv_files = sorted(list(filter(lambda x: x.endswith('.csv') and extract_ts(x) in timestamps, log_files)), key=extract_ts)
json_files = sorted(list(filter(lambda x: x.endswith('.json') and extract_ts(x) in timestamps, log_files)), key=extract_ts)

In [8]:
len(csv_files)

40

In [9]:
def json_file(method, fn):
    return f'log/{fn}_{method}_params.json'

def log_file(method, fn):
    return f'log/{fn}_experiment_{method}_logs.csv'

In [10]:
files

['img/Iteration 3/griewank_efga_summary_plot_1680551765.849279.png',
 'img/Iteration 3/schwefel_efga_summary_plot_1680291367.400345.png',
 'img/Iteration 3/griewank_gendered_summary_plot_1680433819.946909.png',
 'img/Iteration 3/ackley_gendered_summary_plot_1680434399.947702.png',
 'img/Iteration 3/ackley_efga_summary_plot_1680289862.152797.png',
 'img/Iteration 3/schwefel_gendered_summary_plot_1680437762.400973.png',
 'img/Iteration 3/ackley_efga_summary_plot_1680290173.003756.png',
 'img/Iteration 3/schwefel_gendered_summary_plot_1680435733.540228.png',
 'img/Iteration 3/rastrigin_gendered_summary_plot_1680437382.830686.png',
 'img/Iteration 3/griewank_gendered_summary_plot_1680436868.664802.png',
 'img/Iteration 3/griewank_gendered_summary_plot_1680436982.691161.png',
 'img/Iteration 3/rastrigin_gendered_summary_plot_1680433943.827462.png',
 'img/Iteration 3/rastrigin_gendered_summary_plot_1680437253.141597.png',
 'img/Iteration 3/rastrigin_gendered_summary_plot_1680437037.723814.pn

In [11]:
records = []

for i in range(len(csv_files)):
    params = None
    with open(json_files[i], 'r') as f:
        params = json.load(f)

    log = pd.read_csv(csv_files[i])
    params['mean_fitness_fn_calls'] =  log.groupby('seed').first().mean()['ncalls']
    params['median_best_fitness'] =  log.groupby('seed').min().median()['best_fitness']
    params['median_final_best_fitness'] =  log.groupby('seed').last().median()['best_fitness']
    params['best_fitness'] = log.groupby('seed').min().min()['best_fitness']
    # params['method'] =  'efga' if 'efga' in files[i] else 'gendered'
    params['std'] = log.groupby('seed').min('best_fitness').std()['best_fitness']
    records.append(params)
records = pd.DataFrame.from_records(records).sort_values('fitness_fn')

In [12]:
records['method'] = pd.NA
records['method'][records.n_partitions.isna()] = 'efga'
records['method'][~records.n_partitions.isna()] = 'gendered'

/tmp/ipykernel_566/2527682468.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['method'][records.n_partitions.isna()] = 'efga'
/tmp/ipykernel_566/2527682468.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records['method'][~records.n_partitions.isna()] = 'gendered'


In [13]:
records['method_comb'] = pd.NA
records.loc[records.method == 'gendered', 'method_comb'] = 'gendered('+ records.n_partitions[records.method == 'gendered'].astype(int).astype(str) + ')'
records.loc[records.method == 'efga', 'method_comb'] = 'efga('+ records.n_terms_params[records.method == 'efga'].astype(int).astype(str) \
    + ',' + records.n_terms_priority[records.method == 'efga'].astype(int).astype(str)+ ')'

In [14]:
# records.to_csv('summary_iteration_3.csv')

In [15]:
(pd.pivot(records[['std', 'fitness_fn', 'median_best_fitness', 'best_fitness','method_comb']]
 .groupby(['method_comb', 'fitness_fn']).agg({'best_fitness': 'min', 'median_best_fitness': 'mean', 'std': 'mean'}).reset_index(), index='fitness_fn', columns='method_comb')).T.to_csv('summary_iteration_3.csv')

In [17]:
(pd.pivot(records[['std', 'fitness_fn', 'median_best_fitness', 'best_fitness','method_comb']]
 .groupby(['method_comb', 'fitness_fn']).agg({'best_fitness': 'min', 'median_best_fitness': 'mean', 'std': 'mean'}).reset_index(), index='fitness_fn', columns='method_comb')).T.to_csv('summary_iteration_3.csv')